## Installation and imports

In [ ]:
!pip install datasets
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (AutoConfig,
                          AutoTokenizer,
                          AutoModelForTokenClassification,
                          DataCollatorForTokenClassification,
                          TrainingArguments,
                          Trainer,
                          pipeline)
from seqeval.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/machine_learning')
print(f'Working directory: {os.getcwd()}')

Mounted at /content/drive
Working directory: /content/drive/My Drive/machine_learning


In [ ]:
base_model = 'xlm-roberta-base'
trained_model = 'xlm-roberta-base-for-ner'

## Preparing the dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
# stretch the tokenizer's legs
random_sample_text = 'This is going to be a smashing named entity recognition model'
tokens = tokenizer(random_sample_text).tokens()
input_ids = tokenizer.encode(random_sample_text, return_tensors="pt")
pd.DataFrame([tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Tokens,<s>,▁This,▁is,▁going,▁to,▁be,▁a,▁sma,shing,▁na,med,▁enti,ty,▁recognition,▁model,</s>
Input IDs,0,3293,83,7730,47,186,10,44446,54700,24,4806,77630,939,230466,3299,2


In [ ]:
data = pd.read_csv(r'model_ner_dataset.csv', encoding='latin-1')
data['Sentence #'] = data['Sentence #'].ffill()
data = data.dropna(how='any')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [ ]:
grouped_data = data.groupby('Sentence #').agg({'Word': list, 'Tag': list}).reset_index()
dataset = Dataset.from_pandas(grouped_data)
print(dataset)
print(dataset[0]['Word'])

Dataset({
    features: ['Sentence #', 'Word', 'Tag'],
    num_rows: 47959
})
['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']


In [ ]:
tags = set(data['Tag'])
index2tag = {idx: tag for idx, tag in enumerate(tags)}
tag2index = {tag: idx for idx, tag in enumerate(tags)}
print(index2tag)

{0: 'I-per', 1: 'I-gpe', 2: 'B-per', 3: 'I-eve', 4: 'I-nat', 5: 'B-art', 6: 'I-tim', 7: 'B-nat', 8: 'O', 9: 'I-art', 10: 'B-geo', 11: 'B-gpe', 12: 'I-geo', 13: 'B-tim', 14: 'I-org', 15: 'B-eve', 16: 'B-org'}


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["Word"],
                                 truncation=True,
                                 is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["Tag"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(tag2index[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
dataset = dataset.map(tokenize_and_align_labels,
                      batched=True,
                      remove_columns=['Sentence #', 'Word', 'Tag'])
print(dataset)

Map:   0%|          | 0/47959 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 47959
})


In [ ]:
dataset = dataset.train_test_split(test_size=0.1, shuffle=True)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 43163
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4796
    })
})


## Defining a metric for evaluation

In [ ]:
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []
    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])
        labels_list.append(example_labels)
        preds_list.append(example_preds)
    return preds_list, labels_list

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

## Training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
model_config = AutoConfig.from_pretrained(base_model,
                                          num_labels=len(index2tag),
                                          id2label=index2tag, label2id=tag2index)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(base_model,
                                                        config=model_config).to(device)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir=trained_model,
    log_level='error',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    disable_tqdm=False,
    push_to_hub=False,
    report_to='none',
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

<ipython-input-19-2a523b188464>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train() # let's goooo

Epoch,Training Loss,Validation Loss,F1
1,0.110500,0.096917,0.827213
2,0.088200,0.086352,0.837567
3,0.075500,0.084975,0.842893


TrainOutput(global_step=5397, training_loss=0.10497294785204653, metrics={'train_runtime': 1851.4327, 'train_samples_per_second': 69.94, 'train_steps_per_second': 2.915, 'total_flos': 3703655974240212.0, 'train_loss': 0.10497294785204653, 'epoch': 3.0})

## Evaluate the trained model

In [ ]:
def tag_text(text):
    # Get tokens with special characters
    tokens = tokenizer(text).tokens()
    # Encode the sequence into IDs
    input_ids = tokenizer(text, return_tensors='pt').input_ids.to(device)
    # Get predictions as distribution over 7 possible classes
    outputs = model(input_ids)[0]
    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)
    # Convert to DataFrame
    preds = [index2tag[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame(zip(tokens, preds), columns=['Tokens', 'Tags'])

In [ ]:
text = 'Aside from an unfortunate incident with a pig, \
David Cameron successfully guided the United Kingdom \
through a turbulent period.'
print(text)
tag_text(text)

Aside from an unfortunate incident with a pig, David Cameron successfully guided the United Kingdom through a turbulent period.


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
Tokens,<s>,▁A,side,▁from,▁an,▁un,fortuna,te,▁incident,▁with,...,d,▁the,▁United,▁Kingdom,▁through,▁a,▁turbulent,▁period,.,</s>
Tags,O,O,O,O,O,O,O,O,O,O,...,O,O,B-geo,I-geo,O,O,O,O,O,O


## Use for predictions

Run at least the first header above (installation and imports). Then:

In [ ]:
trained_model = r'xlm-roberta-base-for-ner/checkpoint-5397'
texts = ['David Cameron was a pretty good politician', 'But Ed Milliband did a good job at opposition too']

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(trained_model)
tokenizer = AutoTokenizer.from_pretrained(trained_model)

In [ ]:
pipe = pipeline('ner', model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [ ]:
pipe(texts)

[[{'entity': 'B-per',
   'score': np.float32(0.99710983),
   'index': 1,
   'word': '▁David',
   'start': 0,
   'end': 5},
  {'entity': 'I-per',
   'score': np.float32(0.9950244),
   'index': 2,
   'word': '▁Cameron',
   'start': 6,
   'end': 13}],
 [{'entity': 'B-per',
   'score': np.float32(0.9927408),
   'index': 2,
   'word': '▁Ed',
   'start': 4,
   'end': 6},
  {'entity': 'I-per',
   'score': np.float32(0.99251384),
   'index': 3,
   'word': '▁Milli',
   'start': 7,
   'end': 12},
  {'entity': 'I-per',
   'score': np.float32(0.9923205),
   'index': 4,
   'word': 'band',
   'start': 12,
   'end': 16}]]